In [1]:
import sys
import os
from pathlib import Path

from openai import OpenAI
from dotenv import dotenv_values

# Add the parent directory to sys.path so Python can find the Utils module
sys.path.append(os.path.abspath('..'))

from Utils import (get_guidance_and_feedback, get_questions_and_answers, print_bot_response, create_content)

# Load the API key from the .env file
config = dotenv_values('../.env')
client = OpenAI(api_key=config['OPENAI_API_KEY'])
tutor_agent_id = config['TUTOR_AGENT_ID']

Generate the content for our prompt message to our bot.  Our prompt must look like the input examples we provided to our assistant in its instructions.  For this toy example we are using JSON files of question data and student answers as a store.

In [ ]:
q_path = Path('Data/question_store.json')
a_path = Path('Data/student_answers.json')
questions, answers = get_questions_and_answers(q_path, a_path)

Having loaded the question and answer data we can now create input content.

In [ ]:
question_id = "Q2"
student_id = "student_2"

question = questions[question_id]
student_answer = answers[question_id][student_id]

content = create_content(question, student_answer)
print(content)

Now we start a new conversation thread and send our content as a messages to our assistant.  This can take some time to run.

In [ ]:
thread, messages = get_guidance_and_feedback(
    client, content, tutor_agent_id, thread_id=None
)

Now we have the messages we can print out the bot's text response which is embedded within the messages.

In [ ]:
response = print_bot_response(client, messages)

Now we have started the thread, we can continue if we want the chat with the bot.

In [ ]:
follow_up = "I am not sure about the answer. Can you provide more guidance?"

thread, messages = get_guidance_and_feedback(
    client, follow_up, tutor_agent_id, thread_id=thread
)

print_bot_response(client, messages)